## Partie 2

### Question 1

La restriction du problème à un seul véhicule le simplifie beaucoup. On peut le traiter ici de façon analityque. 
Comme nous ne considérons qu'une seule voiture, on peut réduire le cadre du problème en posant l'origine des temps à 0, le moment où la charge commence, et le temps $t_{f}$ où la charge finie.
On va ensuite discrétiser le problème. On choisi un entier $n$, qui va définir le pas: $dt = t_{f} / n$. Le choix de $n$ est donc motivé par le choix de la finesse du pas: plus on souhaite un pas fin, plus $n$ doit être grand. Dans la suite, on notera $t_{i} = i * dt$, et l'on a donc $t_{0} = 0$ et $t_{n} = t_{f}$.
On discrétise ensuite les coûts: le vecteur des coûts est un vecteur de $\mathbb{R}^{n}$ qu'on notera $C$ avec $c_{i} = \frac{1}{dt}\int_{t_{i}}^{t_{i}+dt} c(u)du$ si la fonction coût est au début continue. 
Si le coût est déjà discret, on adapte $n$ afin que $dt$ soit égale au pas de discrétisation des coûts.
Dans les deux cas, on a un vecteur de coût $C$ et un pas $dt$.

La fonction coût est alors $f(X) = dt * <X,C>$ avec $X$ le vecteur des puissances délivrées à la voiture: $x_{i}$ est la puissance délivrée à la voiture entre $t_{i}$ et $t_{i+1}$. Les contraintes que l'on a précédemment définies peuvent alors être exprimées par:

-$c_{1}(X) = X$ et $c_{1}(X) \leq P$ où $P$ est la puissance maximale que le réseau peut délivrer

-$c_{2}(X) = |X| - X$ et $c_{2} \leq 0$ avec $|X| = (|x_{i}|)$

-$c_{3}(X) = E - <V1,X>$ et $c_{3} = 0$ avec $E$ l'énergie qu'on doit délivrer à la voiture pendant le chargement et $V1 = (1)$ un vecteur composé de $1$


On peut maintenant étudier le problème de minimisation de $f$ sous ces contraintes. L'ensemble défini par les contraintes est un convexe compact, la fonction coût est linéaire donc continue et convexe, on à ainsi l'existence d'un minimum. 
Cependant, ce minimum n'est pas unique dans le cas général. Si l'on prend les paramètres suivant: $n = 1, dt = 1, P = 1, C = (1, 1), E = 1$ alors on voit que les solutions $(1, 0)$ et $(0, 1)$ sont toutes les deux valables.

Pour minimiser $f$, on propose un algorithme simple:

1) On note $I = {i \in \mathbb{N}, 0 \leq i \leq n}$ et $X$ un vecteur à $n$ composantes initialisé à $(0, ..., 0)$. On rappelle que $E$ est la quantité d'énergie à délivrer à la voiture.

2) Tant que $E != 0$ ou $I != \emptyset$:

2) 1) On fixe $j = argmin(C_{i}), i \in I$. Si plusieurs choix sont possibles, le choix est fait arbitrairement parmi ceci.

2) 2) On pose $x_{j} = min(P, E/dt)$. 

2) 3) On remplace $I$ par $I - j$ et $E$ par $E - x_{j}$

3) Le vecteur $X$ correspond à un minimum de $f$.

Cet algorithme permet de respecter les contraintes établies, sauf si $E$ est trop grand, et que la charge demandée ne peut pas être effectuée dans le temps imparti.


### Question 2

Commençons par le pas $dt$. Il est probable que l'interface utilisateur permette de chosir le moment de récupération de la voiture à la minute près. On prendra donc $dt$ = 60s.

Pour le coût de l'électricité, on trouve sur internet un prix de 0.134 euros/kWh aux heures creuses et 0.178 euros/kWh aux heures pleines, ce qui donne $C_{-}$ = 3.72e-5 euros/J et $C_{+}$ = 4.94e-5 euros/J.

Pour la quantité d'énergie nécessaire pour une charge, on considère qu'un conducteur averti s'arrête de rouler quand il est à 10% de son autonomie totale. Si l'on prend un Zoe comme modèle de voiture électrique, parce qu'elle est fabriquée en France et d'une jolie couleur bleue, on voit que sa batterie à un capacité de 52kWh ou 1.872e8 J. On va donc prendre $E$ = 1.68e8 J

Pour le temps de charge, cette même Zoe prend 9h25 pour se charger entièrement. On prendra donc 9h pour faire bonne mesure.

In [3]:
import numpy as np

##Implémentation

def findMin(C, I):
    """Renvoie l'élément j de I qui minimise C_j"""
    j = I.pop()
    I.add(j)
    currentMin = C[j]
    for i in I:
        if C[i] < currentMin:
            currentMin = C[i]
            j = i
    return j

def oneCarSolve(C, dt, E, P):
    """Implémentation de l'algorithme décrit ci-dessus. 
    Renvoie un tuple composé d'un booléen et du vecteur des puissances délivrées. Le booléen est True si l'algorithme converge."""
    n = C.shape[0]
    X = np.zeros((n))
    I = set(range(n))
    while (E != 0) or (I != set()):
        j = findMin(C, I)
        X[j] = min(P, E/dt)
        I = I.remove(j)
        E = E - dt*X[j]
    return (E == 0), X

##Données de test

dt = 1 #On prends le temps en minute
E = 1.68 * 100000 #On prends l'énergie en kJ
Cm = 3.72 * 0.01
Cp = 4.94 * 0.01

## Partie 3


### Question 3

Pour repasser au cadre de plusieurs voitures et considérer l'algorithme de décomposition/coordination, il nous faut remodeler les contraintes et ajouter la quatrième. 

Commençons par rappeler les notations: $n$ est le nombre de voitures et $E_{i}$ est l'énergie que l'on doit délivrer à la voiture $i$. La voiture $i$ charge de $t_{i}^{0}$ à $t_{i}^{1}$. On travaille ici en énergie pour simplifier les notations et les calculs. 

On pose $t_{0} = min(t_{i}^{0})$ et $t_{f} = max(t_{i}^{1})$. On note $N$ le nombre d'intervalles de temps, $T = t_{f} - t_{0}$ et $dt = \frac{T}{N}$. On travaille donc dans l'espace des vecteurs $\mathbb{R}^{N}$. 

Le vecteur des coûts est $F$. On peut poser comme nouvelle origine des temps 0, ce qui permet d'éliminer $t_{0}$ des notations. Dans la suite, on considère que tous less temps ont été décalés de $t_{0}$, mais on ne les renomme pas.

On note ensuite, pour une voiture $i$, $B_{i}$ le vecteur colonne à $N$ lignes où $b_{i,j}$ vaut $1$ si la voiture est en charge entre $dt * j$ et $dt * (j+1)$, et $0$ sinon. 

Pour chaque voiture $i$, on note $P_{i}$ le vecteur des puissances délivrées, et on synthétise tout cela dans la variable $P = (P_{i}^{T})$ qui est donc une matrice $n*N$.
Ce qui nous amène à la fonction coût: $g(X) = dt*U^{T}PF$ où $U = (1,...,1)$ est un vecteur colonne à $n$ lignes.

On va séparer ces fonctions coût selon les lignes de $P$. On note ainsi $g_{i}(P_{i}^{T}) = dt*\sum_{j=1}^{N}f_{j}*p_{i,j}$, et on a bien $g(X) = \sum^{n}_{i=1}g_{i}(P_{i}^{T})$.

Il nous faut maintenant adapter les contraites. On se heurte alors à un problème: si l'on choisi de découper la matrice $P$ selon les colonnes, il est impossible d'implémenter la contrainte sur l'énergie délivrée à la voiture, car cette containte nécessite la connaissance d'une ligne entière. Réciproquement, si l'on découpe $P$ selon les colonnes, on ne peut pas implémenter la contrainte sur la puissance maximale délivrée par le réseau, car cette contrainte nécessite la connaissance d'une colonne. Ce problème est du au fait que l'algorithme de décomposition/coordination est valable si les différents problèmes ne se contaignent pas les uns les autres: puisqu'on décompose les contraintes et la fonction coût, on perd de l'information quand à la simultanéité des problèmes. 

Pour résoudre ce problème, on se poropose de passer à un découpage en lignes de $P$ et considérer qu'entre un temps $dt * j$ et $dt * (j+1)$, on ne délivre pas plus de $\frac{Pmax}{\sum^{n}_{i=1}b_{i,j}}$ à une voiture, autrement dit on ne délivre pas plus la puissance maximale divisée par le nombre de voiture en charge. Bien que ça ne soit pas optimal, ceci permet d'éliminer la contrainte qui porte sur les colonnes.

On se retrouve donc avec cette contrainte, qui doit rester en dessous de 0, avec $D = \frac{1}{n}\sum_{i=1}^{n}B_{i}$ un vecteur colonnes à $N$ lignes et $U = (1,...,1)$ un vecteur colonne à $N$ lignes.

$c_{i}(P_{i}^{T}) = ((P_{i} - Pmax*D)^{T}, -P_{i}^{T}, P_{i}^{T} * (1-B_{i}), dt*X_{i}^{T}*U - E_{i})$

Comme on le voit, le vecteur des contraintes concatène toutes les contraintes, et avec 2 contraintes d'inégalités et 2 d'égalités, on obtient un vecteur ligne à $2N+2$ composantes.

Avec un compromis sur la contrainte de puissance, on peut donc implémenter cet algorithme.

### Question 4

Voici l'implémentation dans le cas de $n$ véhicules, qu'on peut appliquer au cas de deux véhicules en posant $n = 2$.

Pour la minimisation individuelle, on utilise la méthode de descente de gradient stochastique.

In [4]:
##On considère qu'on a déja F le vecteur des coûts de taille N*1, le pas dt, la puissance max Pmax.

class Voiture:
    
    def __init__(j0, j1, E):
        self.j0 = j0 #t_j0 est le temps auquel la charge commence
        self.j1 = j1 #t_j1 est le temps auquel la charge fini
        self.E = E
        
def formalizeData(voitureArray, F):
    """Prépare la matrice X, B, E, et D commme décrite ci-dessus pour l'algorithme de décomposition / coordination"""
    n, N = len(voitureArray), len(F)
    B = np.zeros((n,N), dtype=np.bool)
    E = np.zeros((n))
    P = np.zeros((n,N))
    for i in range(n):
        v = voitureArray[i]
        for j in range(v.j0, v.j1):
            B[i,j] = 1
        E[i] = v.E
    D = np.zeros((N))
    for j in range(N):
        D[j] = np.sum(B[:,j])
    D = D / n
    return B, P, E, D

def minimiseIndividuel(F, B, D, P, E, dt, Pmax, mul, tries = 10000, dx = 0.01):
    g = lambda x: np.dot(F, x) + np.dot(mul, contrainte(B, D, P, E, dt, Pmax))
    n = len(X)
    theta = 1 + random.random() - random.random()
    while tries != 0:
        i = random.randrange(n)
        acc = np.zeros(P.shape)
        acc[i] = dx
        P[i] = P[i] - theta*(f(P + acc) - f(P))/dx
        tries -= 1
    return P

def project(arr):
    """Projète un vecteur de R^n sur R+^n"""
    for i in range(len(arr)):
        if arr[i] < 0:
            arr[i] = 0
    return arr

def contrainte(B, D, P, E, dt, Pmax):
    """Renvoie le vecteur des contraintes comme décrit ci-dessus"""
    L = []
    L = L + list(P - Pmax*D)
    L = L + list(-p)
    L = L + [np.dot(P, (1 - B))]
    L = L + [dt*np.sum(P) - E]
    return L

def minimse(voitureArray, F, dt, Pmax, eps, mul, pas):
    """Renvoie une matrice de taille n*N contenant les puissances délivrées aux voitures en minimisant le coût des recharges.
    On utilise la simplification évoquée ci-dessus pour la contrainte sur Pmax.
    voitureArray contient des éléments de la classe Voiture; dt est le pas de temps; F est le vecteur des coûts;
    Pmax la puissance max; eps l'écart entre deux multiplicateurs pour que l'algorithme s'arrête;
    mul et pas sont les données choisie au début de l'algorithme pour le premier multiplicateur de Lagrange et le pas."""
    B, P, E, D = formalizeData(voitureArray, f)
    n, N = len(voitureArray), len(P)
    old_mul = mul + 2*eps
    while abs(mul - old_mul) > eps:
        new_mul = mul
        for i in range(n):
            P[i] = minimiseIndividuel(F, B[i], D, P[i], E[i], dt, Pmax, mul)
            new_mul += pas*contrainte(B[i], D, P[i], E[i], dt, Pmax)
        old_mul = mul
        mul = project(new_mul)
    return P

Cependant, on voit apparaître un problème: le lagrangien est linéaire. Si on essaye de le minimiser sans contraite, comme la méthode de décomposition/coordination le demande, on ne va pas arriver à un minimum, car il n'en existe pas. Ceci est sans doute du à l'hypothèse que l'on fait sur les puissances. 

Pour palier ce problème, on propose une autre méthode de résolution, basée sur l'algorithme exposé en partie 2.

In [5]:
def minimse2(voitureArray, F, dt, Pmax):
    """Renvoie une matrice de taille n*N contenant les puissances délivrées aux voitures en minimisant le coût des recharges.
    Renvoie également un vecteur ligne, qui note si la voiture i a été chargée comme prévu à la case i.
    On utilise la simplification évoquée ci-dessus pour la contrainte sur Pmax.
    voitureArray contient des éléments de la classe Voiture; dt est le pas de temps; F est le vecteur des coûts;
    Pmax la puissance max."""
    B, P, E, D = formalizeData(voitureArray, F)
    D = Pmax*D
    n, N = len(voitureArray), len(A)
    done = np.zeros((n), dtype = np.bool)
    for i in range(n):
        v = voitureArray[i]
        j0, j1 = v.j0, v.j1
        P[i, j0 : j1] = oneCarSolve2(P[j0 : j1], dt, E[i], D[j0 : j1])
        if (np.sum(P[i]) == (E[i]/dt)):
            done[i] = True
    return P, done

def oneCarSolve(C, dt, E, D):
    """Implémentation de l'algorithme décrit ci-dessus. 
    Renvoie un tuple composé d'un booléen et du vecteur des puissances délivrées. Le booléen est True si l'algorithme converge."""
    n = C.shape[0]
    X = np.zeros((n))
    I = set(range(n))
    while (E != 0) and (I != set()):
        j = findMin(C, I)
        X[j] = min(D[j], E/dt)
        I = I.remove(j)
        E = E - dt*X[j]
    return X